# Navigation - Test

---
For Testing and visualizing the Trained Agents

### 1. Start the Environment

We begin by importing some necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Banana.app"`
- **Windows** (x86): `"path/to/Banana_Windows_x86/Banana.exe"`
- **Windows** (x86_64): `"path/to/Banana_Windows_x86_64/Banana.exe"`
- **Linux** (x86): `"path/to/Banana_Linux/Banana.x86"`
- **Linux** (x86_64): `"path/to/Banana_Linux/Banana.x86_64"`
- **Linux** (x86, headless): `"path/to/Banana_Linux_NoVis/Banana.x86"`
- **Linux** (x86_64, headless): `"path/to/Banana_Linux_NoVis/Banana.x86_64"`

For instance, if you are using a Mac, then you downloaded `Banana.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Banana.app")
```

In [2]:
env = UnityEnvironment(file_name="Banana_Linux/Banana.x86_64")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
env.brain_names

['BananaBrain']

In [4]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

The simulation contains a single agent that navigates a large environment.  At each time step, it has four actions at its disposal:
- `0` - walk forward 
- `1` - walk backward
- `2` - turn left
- `3` - turn right

The state space has `37` dimensions and contains the agent's velocity, along with ray-based perception of objects around agent's forward direction.  A reward of `+1` is provided for collecting a yellow banana, and a reward of `-1` is provided for collecting a blue banana. 

Run the code cell below to print some information about the environment.

In [5]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space 
state = env_info.vector_observations[0]
print('States look like:', state)
state_size = len(state)
print('States have length:', state_size)

Number of agents: 1
Number of actions: 4
States look like: [1.         0.         0.         0.         0.84408134 0.
 0.         1.         0.         0.0748472  0.         1.
 0.         0.         0.25755    1.         0.         0.
 0.         0.74177343 0.         1.         0.         0.
 0.25854847 0.         0.         1.         0.         0.09355672
 0.         1.         0.         0.         0.31969345 0.
 0.        ]
States have length: 37


### 3. Run The Agents

In [6]:
import gym

import random
import torch
import numpy as np
from collections import deque
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
from brains import FullyConnectedBrain
from brains import DuelingFullyConnectedBrain
from agents import QAgentGeneric

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### Plotting

In [9]:
def plot_scores(scores, score_label="Average Score (100 Steps)", N = 25, plot_title=""):
  """Plot Scores with Moving Average
  
  Params
    ======
        scores (list): list of scores
        score_label (str): label for scores graph
        N (int): moving average window size
        plot_title (str): title of the plot
  """
  sns.set(style='whitegrid', context='notebook', palette='icefire')
  
 
  moving_average = np.convolve(scores, np.ones((N,))/N, mode='valid')

  fig = plt.figure(figsize=(12, 8))
  ax = fig.add_subplot(111)
  plt.plot(np.arange(len(scores)), scores, label=score_label)
  plt.plot(np.arange(len(moving_average))+(N/2), moving_average, linestyle='--', label="Moving Average")
  plt.ylabel('Score')
  plt.xlabel('Episode #')
  plt.axhline(y=0, color='r', linestyle='-.', label="ZERO")
  plt.axhline(y=13, color='green', linestyle='-.', label="Weakly Solved Threshold")
  plt.axhline(y=17, color='gold' , linestyle='-.', label="Advanced Solved Threshold")
  plt.legend(loc='lower center', bbox_to_anchor=(0.5, -0.25), ncol=3)
  plt.title(plot_title)
  plt.show()

In [10]:
def plot_average_scores(list_of_scores, labels, N = 25, plot_title=""):
  """Plot Average Scores for Multiple Algorithms
  
  Params
  ======
        list_of_scores (list): list of list of scores!
        labels (list): list of labels corresponding to each score
        N (int): moving average window size
        plot_title (str): title of the plot
  """
  sns.set(style='whitegrid', context='notebook', palette='nipy_spectral')
  fig = plt.figure(figsize=(12, 8))
  ax = fig.add_subplot(111)
  
  for i, scores in enumerate(list_of_scores):
    moving_average = np.convolve(scores, np.ones((N,))/N, mode='valid')
    plt.plot(np.arange(len(moving_average))+(N/2), moving_average, label=labels[i])
    
  plt.axhline(y=0, color='r', linestyle='-.', label="ZERO")
  plt.axhline(y=13, color='green', linestyle='-.', label="Weakly Solved Threshold")
  plt.axhline(y=17, color='gold' , linestyle='-.', label="Advanced Solved Threshold")
  plt.ylabel('Score')
  plt.xlabel('Episode #')
  plt.legend(loc='lower center', bbox_to_anchor=(0.5, -0.25), ncol=3)
  plt.title(plot_title)
  plt.show()

### Runner

In [11]:
def dqn(agent, n_episodes=1000, max_t=1000, eps_start=1.0, eps_end=0.01, eps_decay=0.995, quiet=False, save_solved=True, checkpoint_name='checkpoint', min_best = 15, save_best=False):
    """Deep Q-Learning.
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    scores = []                                           # list containing scores from each episode
    scores_window = deque(maxlen=100)                     # last 100 scores
    eps = eps_start                                       # initialize epsilon
    last_best = min_best
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]  # reset the environment
        state = env_info.vector_observations[0]            # get the current state
        score = 0                                          # initialize the score
        
        for t in range(max_t):
            action = agent.act(state, eps)
            env_info = env.step(action)[brain_name]        # send the action to the environment
            next_state = env_info.vector_observations[0]   # get the next state
            reward = env_info.rewards[0]                   # get the reward
            done = env_info.local_done[0]                  # see if episode has finished
            agent.step(state, action, reward, next_state, done) # UPDATE THE NETWORK
            state = next_state                             # roll over the state to next time step
            score += reward                                # update the score
            if done:
                break
        scores_window.append(score)                        # save most recent score
        scores.append(score)                               # save most recent score
        eps = max(eps_end, eps_decay*eps)                  # decrease epsilon
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="")
        if i_episode % 100 == 0:
            if not quiet: print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
        
        average_score = np.mean(scores_window)
        if save_solved and average_score>=20.0:
          if not quiet: print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_window)))
          torch.save(agent.qnetwork_local.state_dict(), f'trained_agents/{checkpoint_name}-{i_episode}-solved.pth')
        
        if save_best and average_score>last_best:
          torch.save(agent.qnetwork_local.state_dict(), f'trained_agents/{checkpoint_name}-Best.pth')
          if not quiet: print(f'Saved Best with Average Score of {average_score}\n')
          last_best = average_score

    return scores

### Vanilla DQN

In [12]:
lb = FullyConnectedBrain(state_size=37, action_size=4, seed=0, hidden_sizes=[128, 64, 16]).to(device) # Local Network
tb = FullyConnectedBrain(state_size=37, action_size=4, seed=0, hidden_sizes=[128, 64, 16]).to(device) # Target Network

vanilla_dqn_agent = QAgentGeneric(state_size=37, action_size=4, seed=0, update_rule="dqn", local_brain=lb, target_brain=tb)

### Double DQN

In [13]:
lb = FullyConnectedBrain(state_size=37, action_size=4, seed=0, hidden_sizes=[128, 64, 16]).to(device) # Local Network
tb = FullyConnectedBrain(state_size=37, action_size=4, seed=0, hidden_sizes=[128, 64, 16]).to(device) # Target Network

double_dqn_agent = QAgentGeneric(state_size=37, action_size=4, seed=0, update_rule="double_dqn", local_brain=lb, target_brain=tb)

### Dueling DQN (Vanilla)

In [14]:
lb = DuelingFullyConnectedBrain(state_size=37, action_size=4, seed=0, shared_hidden = [128, 64, 16], value_head=[32, 16], advantage_head=[32, 32]).to(device)  # Local Network
tb = DuelingFullyConnectedBrain(state_size=37, action_size=4, seed=0, shared_hidden = [128, 64, 16], value_head=[32, 16], advantage_head=[32, 32]).to(device)  # Target Network

dueling_dqn_agent = QAgentGeneric(state_size=37, action_size=4, seed=0, update_rule="dqn", local_brain=lb, target_brain=tb)

### Dueling Double DQN

In [15]:
lb = DuelingFullyConnectedBrain(state_size=37, action_size=4, seed=0, shared_hidden = [128, 64, 16], value_head=[32, 16], advantage_head=[32, 32]).to(device)  # Local Network
tb = DuelingFullyConnectedBrain(state_size=37, action_size=4, seed=0, shared_hidden = [128, 64, 16], value_head=[32, 16], advantage_head=[32, 32]).to(device)  # Target Network

dueling_dobule_agent = QAgentGeneric(state_size=37, action_size=4, seed=0, update_rule="double_dqn", local_brain=lb, target_brain=tb)

### Testing our Trained Agent

In [16]:
def load_agent(index):
  """returns the selected agent, with best weights"""
  available_agents = [vanilla_dqn_agent, double_dqn_agent, dueling_dqn_agent, dueling_dobule_agent]
  best_weights = ['dqn_vanilla-Best.pth', 'double_dqn-Best.pth', 'dueling_dqn-Best.pth', 'dueling_double_dqn-Best.pth']
  
  available_agents[index].qnetwork_local.load_state_dict(torch.load(f'trained_agents/{best_weights[index]}'))

  return available_agents[index]

In [23]:
def run_agent(agent, N=100):
  """runs the env with selected agent """
  env_info = env.reset(train_mode=False)[brain_name]  # reset the environment
  state = env_info.vector_observations[0]             # get the current state
  score = 0                                           # initialize the score
  
  for j in range(N):
    action = int(agent.act(state))
    env_info = env.step(action)[brain_name]           # send the action to the environment
    next_state = env_info.vector_observations[0]      # get the next state
    reward = env_info.rewards[0]                      # get the reward
    done = env_info.local_done[0]                     # see if episode has finished
    score += reward                                   # update the score
    state = next_state                                # roll over the state to next time step

    if done:
      break
      
  return score

In [25]:
# Run DQN with trained Agent, 
total_score = run_agent(load_agent(3)) 

print(total_score)

8.0


### Finished? Turn off the lights and Close the ENV

In [26]:
env.close() 